<a href="https://colab.research.google.com/github/chaitraDev/GenAI-for-marketing/blob/main/Cognizant_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [ ]:
path = "/content/drive/MyDrive/Cognizant/bank_transactions.csv"
bank_df = pd.read_csv(path)
bank_df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Handling Outliers

In [ ]:
# DOBs 1800 are wrong
bank_df[bank_df["CustomerDOB"]=="1/1/1800"]

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)


In [ ]:
bank_df["CustomerDOB"].replace("1/1/1800","1-1-1998",inplace=True)

In [ ]:
# the customer has wrong data
wrongs = bank_df[bank_df["CustomerDOB"]=="19-01-21"].index
bank_df.drop(wrongs,inplace=True)

# Dropping some rows whose customerID isn't repeated

In [ ]:
#@title checking the count of transaction for each customer
counts = bank_df["CustomerID"].value_counts()
counts = counts.reset_index()
counts.head()

,CustomerID,count
0,C5533885,6
1,C6624352,6
2,C1736254,6
3,C6735477,6
4,C1113684,6


In [ ]:
#@title filtering the rows with count of 1
indexes = counts[counts["count"]==1].index
indexes

Index([143612, 143613, 143614, 143615, 143616, 143617, 143618, 143619, 143620,
       143621,
       ...
       884255, 884256, 884257, 884258, 884259, 884260, 884261, 884262, 884263,
       884264],
      dtype='int64', length=740653)

In [ ]:
#@title dropping random rows with count of 1
to_drop = [np.random.choice(indexes) for _ in range(7_00_000)]
print(len(to_drop),len(set(to_drop)))
bank_df.drop(set(to_drop),inplace=True)
bank_df.shape

700000 452907


(595660, 9)

In [ ]:
bank_df.to_csv("bank.csv",index=False)

# Separating Customer and Transaction data

In [ ]:
cust_cols = ["CustomerID","CustomerDOB","CustGender","CustLocation","CustAccountBalance"]

In [ ]:
#@title Transactions Data
transactions_df = bank_df.drop(columns=cust_cols[1:])
print(transactions_df.columns)
transactions_df.to_csv("transactions.csv",index=False)

Index(['TransactionID', 'CustomerID', 'TransactionDate', 'TransactionTime',
       'TransactionAmount (INR)'],
      dtype='object')


In [ ]:
#@title Customer Data
cust_df = bank_df[cust_cols]
print(cust_df.columns)

# Transaction has multiple values for some customers, dropping them from customer table
cust_df.drop_duplicates(subset="CustomerID",keep="first",inplace=True)
print(cust_df["CustomerID"].value_counts())
cust_df.to_csv("customers.csv",index=False)

In [ ]:
cust_df.isnull().sum()

,0
CustomerID,0
CustomerDOB,1747
CustGender,540
CustLocation,87
CustAccountBalance,1195
